In [1]:
import numpy as np
import scipy.linalg as la
import pandas as pd

# Extrair informações da MIP

In [2]:
#Produção total
V = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Recursos',skiprows=5,skipfooter=4,usecols=[*range(11,78)],header=None)
V = np.transpose(V)
#Impostos
Imp = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Recursos',skiprows=5,skipfooter=4,usecols=[8],header=None)
#Consumo intermediário produção nacional
Un = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Nacional',skiprows=5,skipfooter=4,usecols=[*range(2,69)],header=None)
#Demanda Final Nacional
Fn = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Nacional',skiprows=5,skipfooter=4,usecols=[*range(70,76)],header=None)

#Consumo intermediário produção importados
Um = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(2,69)],header=None)
#Demanda Final Nacional importados
Fm = pd.read_excel('./dados/MIP_2019_67_PCE.xlsx',sheet_name='Usos Importado',skiprows=5,skipfooter=4,usecols=[*range(70,76)],header=None)


(67, 67)

# Extrair informações da TRU


In [167]:
#Dados da TRU
planilha = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2019.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
VAB = planilha.iloc[0,:].to_numpy()#Valor adicionado bruto
W = planilha.loc[1,:].to_numpy()#Remuneracoes
RMB = planilha.loc[8,:].to_numpy()#Rendimento misto bruto
Out_Imp = planilha.loc[10,:].to_numpy()# outros impostos e subsidios
Sub = planilha.loc[11,:].to_numpy()# outros impostos e subsidios
Imp_ou_sub = Out_Imp + Sub
VBP = planilha.iloc[12,:].to_numpy()#Valor bruto de produção
o = planilha.loc[13,:]# ocupações

#Matriz para agregação de 68 atividades em 67 da TRU
agg_68_67 = np.concatenate((np.eye(67,41),np.concatenate((np.zeros((40,27)),np.eye(27,27)))),axis=1)
#Transformação nas matrizes para 67 atividades
VAB = np.matmul(agg_68_67,VAB.transpose())
W = np.matmul(agg_68_67,W.transpose())
RMB = np.matmul(agg_68_67,RMB.transpose())
Out_Imp = np.matmul(agg_68_67,Out_Imp.transpose())
Sub = np.matmul(agg_68_67,Sub.transpose())
Imp_ou_sub = np.matmul(agg_68_67,Imp_ou_sub.transpose())
VBP = np.matmul(agg_68_67,VBP.transpose())

o = np.matmul(agg_68_67,o.transpose())




# Derivar informações da MIP

In [4]:
V.shape

(67, 126)

In [5]:
g = np.matmul(V,np.ones(126)) ## Produção total por atividade No caso da MIP estimada, o transporte aquaviário foi fundido com o terrestre de cargas
q = np.transpose(np.matmul(np.transpose(np.ones(67)),V)) ## Produção total por produto
D = np.matmul(V,np.linalg.inv(np.diag(q))) # market share

Bn = np.matmul(Un,np.linalg.inv(np.diag(g))) # consumo intermediário nacional
A = np.matmul(D,Bn) # coeficientes técnicos intersetoriais nacionais

fn = np.matmul(Fn,np.transpose(np.ones(6))) # soma da demanda final
e = np.matmul(D,fn) #Demanda final ( nacional )

Bm = np.matmul(Um,np.linalg.inv(np.diag(g))) # consumo intermediário importado

A_plus = np.matmul(D,Bn+Bm) # coeficientes técnicos ampliada ( contém insumos nacionais e importados )
L = np.linalg.inv(np.eye(A.shape[0])-A) # leontief

l=np.matmul(np.linalg.inv(np.diag(g)),o)#Postos de trabalho por unidade (Milhão) de produção 



# Cálculo em um modelo fechado

In [182]:
sum(W)

3217680.0

In [183]:
cr =  W/VBP #Coeficientes de remuneração do trabalho ( renda )
ch = Fn.loc[:,Fn.columns[3]]/sum(W)

In [187]:
Fn.loc[:,Fn.columns[3]].shape

(126,)

# Cálculo dos multiplicadores

In [6]:
w = np.matmul(np.linalg.inv(np.diag(g)),W) # participação de remuneracoes pela produção total, para cada atividade
wa = np.matmul(np.linalg.inv(np.diag(g)),W+RMB) # participação de remuneracoes pela produção total, para cada atividade

y=np.matmul(np.linalg.inv(np.diag(g)),VAB) # participação de valor agregado pela produção total, para cada atividade
tp=np.matmul(np.matmul(np.linalg.inv(np.diag(g)),D),Imp)#Divisão de impostos pelo market share de cada produto

to=np.matmul(np.matmul(np.diag(y),np.linalg.inv(np.diag(VAB))),Imp_ou_sub)#Outros impostos sobre a produção

tn = tp.to_numpy()+to #Total de impostos sobre produção nacional


In [7]:

massa_salarial = np.matmul(w.transpose(),np.matmul(L,e))
pib = np.matmul(np.matmul(y.transpose()+tp.transpose(),L),e.to_numpy())
Valor_Agregado = np.matmul(np.matmul(y.transpose(),L),e)

In [8]:
#Calcular o incremento de uma unidade (milhão) na atividade identificadada por 1 no vetor arr
arr = np.zeros(67)
arr[7]=1

print("Aumento no consumo intermediário importado de {:0.2f} ".format(np.matmul(np.ones(67),np.matmul(np.matmul(np.matmul(D,Bm),L),arr)*1000000)))#aumento de consumo intermediário importado
print("Aumento no valor agregado de produção de {:0.2f} ".format(np.matmul(np.matmul(y.transpose(),L),arr)*1000000))
print("Aumento no PIB de {:0.2f} ".format(np.matmul(np.matmul(y.transpose()+tp.transpose(),L),arr).values[0]*1000000))
print("Aumento nas ocupações (postos de trabalho) de {:0.2f} ".format(np.matmul(np.matmul(l,L),arr)))
print("Aumento na massa salarial de {:0.2f} ".format(np.matmul(np.matmul(w,L),arr)*1000000))
print("Aumento na massa salarial ampliada de {:0.2f} ".format(np.matmul(np.matmul(wa,L),arr)*1000000))

Aumento no consumo intermediário importado de 120591.27 
Aumento no valor agregado de produção de 792550.11 
Aumento no PIB de 855268.03 
Aumento nas ocupações (postos de trabalho) de 22.74 
Aumento na massa salarial de 364504.47 
Aumento na massa salarial ampliada de 508903.16 


# Cálculo dos multiplicadores para trás - Total

In [9]:
mul_prod = np.matmul(np.ones(67).transpose(),L)#Multiplicador para produção
mul_sal = np.matmul(w.transpose(),L)#multiplicador de remunerações
mul_sal_ajus = np.matmul(wa.transpose(),L)#multiplicador de remunerações ajustado
mul_imp = np.matmul(tp.transpose(),L)#multiplicador de impostos
mul_ocup = np.matmul(l.transpose(),L)#Multiplicador de ocupações


In [67]:
#Sensibilidade de dispersão 
67*np.matmul(L,np.ones(67))/np.matmul(np.matmul(np.ones(67).transpose(),L),np.ones(67).transpose())

array([1.95320877, 0.8849815 , 0.69651252, 0.68040063, 1.3324876 ,
       0.74476656, 0.63324047, 0.78577659, 0.67419397, 1.03375876,
       0.72016829, 0.57548693, 0.9344627 , 0.61897335, 0.64846753,
       0.71939865, 1.08230066, 0.70989595, 2.6988964 , 0.74360494,
       1.84468329, 1.12213464, 0.64160004, 0.63043642, 1.17640513,
       0.83178648, 1.1627701 , 0.93767546, 1.05237792, 0.6922444 ,
       0.79339119, 0.80457056, 0.59615961, 0.92319941, 0.60720664,
       0.64935173, 1.1754622 , 2.03885594, 0.79220327, 0.88946402,
       3.77242316, 2.39673717, 0.68087321, 0.69886388, 1.22857202,
       0.629961  , 0.70319396, 0.59713359, 1.02092752, 0.92725092,
       1.05122676, 2.09715558, 1.01135702, 1.8657704 , 0.75779663,
       1.18068677, 0.8272386 , 1.53768834, 0.74882389, 0.71946422,
       0.56116788, 0.6491335 , 0.5518001 , 0.6283275 , 0.65276674,
       0.72042678, 0.55027163])

In [10]:
poder_dispersao = 67*mul_prod / np.matmul(np.matmul(np.ones(67).transpose(),L),np.ones(67).transpose()) #multiplicador de produção normalizado

In [36]:
#Bn = np.matmul(np.linalg.inv(np.diag(g)),D) # consumo intermediário nacional
#Sensibilidade de dispersão (Gosh)


B = np.matmul(np.matmul(np.linalg.inv(np.diag(g)),D),Un) #coeficientes técnicos a jusante
G = np.linalg.inv(np.eye(B.shape[0])-B) #matriz inversa de Gosh
mul_prod_jus = np.matmul(G,np.ones(67))
sensibilidade_dispersao = 67*mul_prod_jus / np.matmul(np.matmul(np.ones(67),G),np.ones(67)) #multiplicador a jusante normalizado


In [37]:
sensibilidade_dispersao

array([0.93030683, 1.029361  , 0.91788326, 1.53798836, 1.27806992,
       0.79089042, 0.91906648, 0.68314297, 0.9855104 , 0.82339206,
       0.858786  , 0.56103768, 1.03898327, 0.65411674, 0.64458467,
       1.06416928, 1.08682512, 1.52708773, 1.42546099, 1.19599248,
       1.53234583, 1.37408888, 0.79346222, 0.73547872, 1.28451042,
       1.23185979, 1.17294908, 1.09316734, 1.19848806, 0.6988382 ,
       0.95760311, 0.76894814, 0.57368135, 1.19427164, 0.64896209,
       0.72741564, 1.4266994 , 1.3272708 , 1.03327052, 0.67592253,
       0.8852717 , 1.24098313, 1.43432743, 1.08321869, 1.21160945,
       0.99507493, 0.6376978 , 0.990807  , 1.85051404, 0.9214078 ,
       1.01785736, 0.99151457, 0.66058229, 1.40246343, 1.0284862 ,
       1.42402375, 1.32560244, 1.21324048, 1.39505735, 0.5632164 ,
       0.53693509, 0.65779988, 0.53449671, 0.60833452, 0.76174988,
       0.6927858 , 0.53305241])

In [43]:
Un.shape

(126, 67)

In [45]:
np.matmul(np.ones(126),Un)/4

2     41021.590076
3     18042.619448
4      2151.736848
5      2448.495716
6     24125.365106
          ...     
64    16004.078570
65    28834.930185
66     4350.493791
67    17568.981811
68        0.000000
Length: 67, dtype: float64

In [68]:
sensibilidade_dispersao[0]

0.9303068328289611

In [71]:
np.matmul(np.ones(67),A)

0     0.395096
1     0.442742
2     0.225313
3     0.481040
4     0.379000
        ...   
62    0.294935
63    0.355873
64    0.373625
65    0.379667
66    0.000000
Length: 67, dtype: float64

In [70]:
np.matmul(A,np.ones(67))

11    1.848847
12    0.419128
13    0.172770
14    0.137852
15    0.336075
        ...   
73    0.002153
74    0.123647
75    0.112678
76    0.172030
77    0.000000
Length: 67, dtype: float64

# Teste de derivação da MIP a partir da TRU ( Andamento )

In [12]:
ConsumoIntermediario = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA
Demanda = pd.read_excel('./dados/nivel_68_2010_2021_xls/68_tab2_2015.xls',sheet_name='VA',skiprows=5,skipfooter=4,usecols=[*range(1,69)],header=None) # planilha VA


In [6]:
Q = np.array([[280,120],[12,8]])
q = np.array([575,20])

print(Q)
print(q)

[[280 120]
 [ 12   8]]
[575  20]


In [14]:
A= np.matmul(Q,np.linalg.inv(np.diag(q)))

In [19]:
eigvector,eigvalues = la.eig(np.matrix.transpose(A))

In [38]:
lambdas = max(eigvector).real
pos_vec = np.argmax(eigvector)

In [39]:
max_eigvectors = eigvalues[:,pos_vec].reshape(2,1)

In [40]:
r = (1-lambdas)/lambdas

In [37]:
lambdas.real

0.8

In [45]:
max_eigvectors/max_eigvectors[0]

array([[ 1.],
       [15.]])

## Importação de dados reais

In [1]:
import pandas as pd


Undefined variable: import
